# Pickle LRG files for Barry
This notebook includes code to ingest all the Y1 blinded data files. Barry has a strict set of data inputs, and everything gets pickled up into this format. This means the underlying clustering measurements etc., can be in any format.

Barry expects that you will read in/specify and pickle:
* The number of correlated datasets. **We set n_data == 1 below, but this would be 2 if we were providing i.e., NGC+SGC data vectors and wanted to consider different bias or polynomials for each cap**
* Pre and post-recon data power spectrum with 5 multipoles (some multipoles can be set to zero if they are not required/measured). **Post-recon data currently set to None as it doesn't have the same redshift binning.**
* N pre and post-recon mock power spectra with 5 multipoles (some can be set to zero if they are not required/measured). **Mocks currently set to None as we don't have these yet**
* Pre and post-recon covariance matrices for the power spectra (some elements/blocks can be set to zero if they are not required/measured). **Pre-recon analytic used for both currently.**
* A fiducial cosmology. **I've assumed the DESI fiducial cosmology**
* A window function convolution/binning matrix (some elements/blocks can be set to the identify matrix if they are not required/measured), corresponding k-binning and integral constraint. Only needed for power spectra.
* A compression matrix to convert the 3 even multipoles to 5 even+odd (can be given as a block identity matrix if you are not measuring odd multipoles). Only needed for power spectra.

Correlation functions are similar but a little simpler (only 3 multipoles, and no window function stuff).

In [1]:
# Import the necessary packages, set up the fiducial cosmology and save the DESI template
import os
import pickle
import numpy as np
import scipy as sp
import pandas as pd
from astropy.io import ascii
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev
from cosmoprimo import PowerSpectrumBAOFilter
from cosmoprimo.fiducial import DESI
from pypower import BaseMatrix, CatalogFFTPower, CatalogFFTCorr, PowerSpectrumMultipoles, PowerSpectrumSmoothWindow, PowerSpectrumSmoothWindowMatrix, PowerSpectrumOddWideAngleMatrix, setup_logging
from pycorr import TwoPointCorrelationFunction, project_to_multipoles

cosmo = DESI()
print(cosmo["Omega_b"]*cosmo["h"]**2, cosmo["Omega_cdm"]*cosmo["h"]**2, cosmo["Omega_m"]*cosmo["h"]**2 - cosmo["Omega_b"]*cosmo["h"]**2)
print(cosmo["A_s"], cosmo["n_s"], cosmo["tau_reio"])
print(np.sum(cosmo["m_ncdm"]))

# Save the default DESI template to a file
k_min = 1e-4
k_max = 5
k_num = 2000
kl = np.logspace(np.log(k_min), np.log(k_max), k_num, base=np.e)
pkz = cosmo.get_fourier().pk_interpolator()
pk = pkz.to_1d(z=0)
pkv = pk(kl)
pknow = PowerSpectrumBAOFilter(pk, engine='wallish2018').smooth_pk_interpolator()
pksmv = pknow(kl)
np.savetxt("./DESI_Pk_template.dat", np.c_[kl, pksmv, pkv/pksmv - 1.0],  fmt="%g %g %g", header="k     pk_smooth     pk_ratio")

0.02237 0.12 0.1206441345126498
2.083e-09 0.9649 0.0544
0.05999991930682943


# Correlation function routines

In [2]:
# Useful utility function to collate some Xi data
def collect_xi_data(pre_files, post_files, pre_cov_files, post_cov_files, pre_name, post_name, zs):

    pre_data = get_xi(pre_files, pre_name) if pre_files is not None else None
    post_data = get_xi(post_files, post_name) if post_files is not None else None
    
    pre_cov = get_xi_cov(pre_cov_files, pre_name) if pre_cov_files is not None else None
    post_cov = get_xi_cov(post_cov_files, post_name) if post_cov_files is not None else None

    pre_mocks, post_mocks = None, None

    split = {
        "n_data": 1,
        "pre-recon data": pre_data,
        "pre-recon cov": pre_cov,
        "post-recon data": post_data,
        "post-recon cov": post_cov,
        "pre-recon mocks": pre_mocks,
        "post-recon mocks": post_mocks,
        "cosmology": {
            "om": cosmo["Omega_m"],
            "h0": cosmo["h"],
            "z": (zs[1]+zs[0])/2.0,
            "ob": cosmo["Omega_b"],
            "ns": cosmo["n_s"],
            "mnu": np.sum(cosmo["m_ncdm"]),
            "reconsmoothscale": 10,
        },
        "name": "DESI Y1 BLIND " + pre_name,
    }
    
    with open(f"./DESI_Y1_BLIND_" + pre_name.lower().replace(" ", "_")+".pkl", "wb") as f:
        pickle.dump(split, f)
        
    return split

# Correlation function
def get_xi(loc, name):
    
    nran = "nran5" if "MGrecsym" in name else "nran4" 
    infile = loc + "xipoles_" + name + "_default_FKP_lin4_njack0_" + nran + "_split20.txt"
    
    xi = pd.read_csv(infile, comment="#", skiprows=0, delim_whitespace=True, header=None, names=["s", "savg","xi0","xi2","xi4"])
    xi = xi.drop(xi[xi["s"] < 20.0].index)
    
    return [xi[["s","xi0","xi2","xi4"]]]

# Correlation function covariance matrix.
def get_xi_cov(loc, name):

    nran = "nran5" if "MGrecsym" in name else "nran4" 
    infile = loc + "xi024_" + name + "_default_FKP_lin4_s20-200_cov_RascalC_Gaussian.txt"
    
    cov = pd.read_csv(infile, comment="#", delim_whitespace=True, header=None).to_numpy()
    
    #plt.imshow(cov/np.sqrt(np.outer(np.diag(cov), np.diag(cov))))
    #plt.show()
    
    # Check the covariance matrix is invertible
    v = np.diag(cov @ np.linalg.inv(cov))
    if not np.all(np.isclose(v, 1)):
        print("ERROR, setting an inappropriate covariance matrix that is almost singular!!!!")

    return cov
    
# Plot the correlation function, for sanity checking
def plot_xi(split, pre=True, post=True):
        
    if pre:
    
        color = ["r", "b", "g"]
        ss = split["pre-recon data"][0]["s"]
        label = [r"$\xi_{0}(k)$", r"$\xi_{2}(k)$", r"$\xi_{4}(k)$"]
        for m, xi in enumerate(["xi0", "xi2", "xi4"]):
            yerr = ss ** 2 * np.sqrt(np.diag(split["pre-recon cov"]))[m * len(ss) : (m + 1) * len(ss)]
            plt.errorbar(
                ss,
                ss ** 2 * split["pre-recon data"][0][xi],
                yerr=yerr,
                marker="o",
                ls="None",
                c=color[m],
                label=label[m],
            )
        plt.xlabel(r"$s$")
        plt.ylabel(r"$s^{2}\,\xi(s)$")
        plt.title(split["name"] + " Prerecon")
        plt.legend(loc='upper right')
        plt.show()
        
    if post:
        
        color = ["r", "b", "g"]
        ss = split["post-recon data"][0]["s"]
        label = [r"$\xi_{0}(k)$", r"$\xi_{2}(k)$", r"$\xi_{4}(k)$"]
        for m, xi in enumerate(["xi0", "xi2", "xi4"]):
            yerr = ss ** 2 * np.sqrt(np.diag(split["post-recon cov"]))[m * len(ss) : (m + 1) * len(ss)]
            plt.errorbar(
                ss,
                ss ** 2 * split["post-recon data"][0][xi],
                yerr=yerr,
                marker="o",
                ls="None",
                c=color[m],
                label=label[m],
            )
        plt.xlabel(r"$s$")
        plt.ylabel(r"$s^{2}\,\xi(s)$")
        plt.title(split["name"] + " Postrecon")
        plt.legend(loc='upper right')
        plt.show()

# Grab all the various datasets!!
This will also plot the datasets, but this has been commented out in the GitHub version for  

In [5]:
# This is a dictionary of all the combinations of dataset that we have and their redshift bins. First array is zmin, second is zmax
tracers = {'BGS_BRIGHT-21.5': [[0.1, 0.4]], 
           'LRG': [[0.4, 0.6], [0.6, 0.8], [0.8, 1.1]], 
           'ELG_LOPnotqso': [[0.8, 1.1], [1.1, 1.6]],
           'QSO': [[0.8, 2.1]]}

# The different sky areas
caps = ["NGC", "SGC", "GCcomb"]

pre_files = "/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v0.1/blinded/xi/smu/"
post_files = "/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v0.1/blinded/recon_sm10/xi/smu/"
pre_cov_files = "/global/cfs/cdirs/desi/users/mrash/RascalC/Y1/blinded/v0.1/"
post_cov_files = "/global/cfs/cdirs/desi/users/mrash/RascalC/Y1/blinded/v0.1/"

# Now loop over each tracer, redshift bin and cap and gather the files. First pre-recon
for t in tracers:
    for i, zs in enumerate(tracers[t]):
        for cap in caps:
            
            # Correlation Function
            pre_name = f"{t}_{cap}_{zs[0]}_{zs[1]}"
            data = collect_xi_data(pre_files, None, pre_cov_files, None, pre_name, None, zs)
            #plot_xi(data, post=False) # Plot the data to check things

# Test we can load the datasets in Barry
This loads in the pickle file using the Barry routines, and then plots the data and a default model. Because I've set marg="full", it will automatically find the best-fit polynomial terms too when plotting, but not the BAO parameters. If this works, a full MCMC BAO fit should work.

In [4]:
import os
import sys
import pickle
import numpy as np

sys.path.append("../../../Barry/")     # Change this so that it points to where you have Barry installed
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4

def plot_xi(data):
            
    color = ["r", "b", "g"]
    ss = split["pre-recon data"][0]["s"]
    label = [r"$\xi_{0}(k)$", r"$\xi_{2}(k)$", r"$\xi_{4}(k)$"]
    for m, xi in enumerate(["xi0", "xi2", "xi4"]):
        yerr = ss ** 2 * np.sqrt(np.diag(split["pre-recon cov"]))[m * len(ss) : (m + 1) * len(ss)]
        plt.errorbar(
            ss,
            ss ** 2 * split["pre-recon data"][0][xi],
            yerr=yerr,
            marker="o",
            ls="None",
            c=color[m],
            label=label[m],
        )
    plt.xlabel(r"$s$")
    plt.ylabel(r"$s^{2}\,\xi(s)$")
    plt.title(split["name"] + " Prerecon")
    plt.legend(loc='upper right')
    plt.show()

for t in tracers:
    for i, zs in enumerate(tracers[t]):
        for cap in caps:
            
            name = f"DESI_Y1_BLIND_{t.lower()}_{cap.lower()}_{zs[0]}_{zs[1]}.pkl"
            dataset = CorrelationFunction_DESI_KP4(
                recon=None,
                fit_poles=[0, 2],
                min_dist=52.0,
                max_dist=150.0,
                realisation='data',
                num_mocks=1000,
                reduce_cov_factor=1,
                datafile=name,
                data_location="./",
            )
            
            model = CorrBeutler2017(
                recon=dataset.recon,
                isotropic=dataset.isotropic,
                marg="full",
                poly_poles=dataset.fit_poles,
                n_poly=4,    # 4 polynomial terms for Xi(s)
            )
            
            model.set_data(dataset.get_data())
            #model.plot(model.get_param_dict(model.get_defaults()))